# Textklassifizierungsaufgabe

Wie bereits erwähnt, konzentrieren wir uns auf eine einfache Textklassifizierungsaufgabe basierend auf dem **AG_NEWS**-Datensatz. Ziel ist es, Nachrichtenüberschriften in eine von vier Kategorien einzuordnen: Welt, Sport, Wirtschaft und Wissenschaft/Technik.

## Der Datensatz

Dieser Datensatz ist im [`torchtext`](https://github.com/pytorch/text)-Modul integriert, sodass wir leicht darauf zugreifen können.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Hier enthalten `train_dataset` und `test_dataset` Sammlungen, die jeweils Paare aus Label (Nummer der Klasse) und Text zurückgeben, zum Beispiel:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Also, drucken wir die ersten 10 neuen Schlagzeilen aus unserem Datensatz aus:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Da Datensätze Iteratoren sind, müssen wir sie in eine Liste umwandeln, wenn wir die Daten mehrmals verwenden möchten:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Tokenisierung

Nun müssen wir Text in **Zahlen** umwandeln, die als Tensoren dargestellt werden können. Wenn wir eine Wortebene-Darstellung möchten, müssen wir zwei Dinge tun:
* einen **Tokenizer** verwenden, um den Text in **Tokens** zu zerlegen
* ein **Vokabular** dieser Tokens erstellen.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Mit Vokabular können wir unsere tokenisierte Zeichenkette leicht in eine Zahlenmenge kodieren:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Bag-of-Words Textdarstellung

Da Wörter Bedeutung repräsentieren, können wir manchmal die Bedeutung eines Textes allein durch die Betrachtung der einzelnen Wörter herausfinden, unabhängig von ihrer Reihenfolge im Satz. Zum Beispiel, wenn wir Nachrichten klassifizieren, deuten Wörter wie *Wetter*, *Schnee* wahrscheinlich auf eine *Wettervorhersage* hin, während Wörter wie *Aktien*, *Dollar* eher auf *Finanznachrichten* hinweisen.

Die **Bag-of-Words** (BoW)-Vektordarstellung ist die am häufigsten verwendete traditionelle Vektordarstellung. Jedes Wort ist einem Vektorindex zugeordnet, und das Vektorelement enthält die Anzahl der Vorkommen eines Wortes in einem bestimmten Dokument.

![Bild, das zeigt, wie eine Bag-of-Words-Vektordarstellung im Speicher dargestellt wird.](../../../../../lessons/5-NLP/13-TextRep/images/bag-of-words-example.png) 

> **Hinweis**: Sie können BoW auch als die Summe aller One-Hot-encodierten Vektoren für einzelne Wörter im Text betrachten.

Im Folgenden finden Sie ein Beispiel, wie man eine Bag-of-Words-Darstellung mit der Scikit Learn Python-Bibliothek erzeugt:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Um den Bag-of-Words-Vektor aus der Vektordarstellung unseres AG_NEWS-Datensatzes zu berechnen, können wir die folgende Funktion verwenden:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Hinweis:** Hier verwenden wir die globale Variable `vocab_size`, um die Standardgröße des Vokabulars festzulegen. Da die Vokabulargröße oft ziemlich groß ist, können wir die Größe des Vokabulars auf die häufigsten Wörter begrenzen. Versuchen Sie, den Wert von `vocab_size` zu verringern und den untenstehenden Code auszuführen, und beobachten Sie, wie sich dies auf die Genauigkeit auswirkt. Sie sollten einen gewissen Rückgang der Genauigkeit erwarten, aber keinen dramatischen, zugunsten einer höheren Leistung.


## Training eines BoW-Klassifikators

Jetzt, da wir gelernt haben, wie man eine Bag-of-Words-Darstellung unseres Textes erstellt, lassen Sie uns einen Klassifikator darauf trainieren. Zunächst müssen wir unser Dataset für das Training so umwandeln, dass alle Positionsvektordarstellungen in Bag-of-Words-Darstellungen konvertiert werden. Dies kann erreicht werden, indem die Funktion `bowify` als Parameter `collate_fn` an den standardmäßigen torch `DataLoader` übergeben wird:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Lassen Sie uns nun ein einfaches Klassifikator-Neuronales Netzwerk definieren, das eine lineare Schicht enthält. Die Größe des Eingabevektors entspricht `vocab_size`, und die Ausgabengröße entspricht der Anzahl der Klassen (4). Da wir eine Klassifikationsaufgabe lösen, ist die endgültige Aktivierungsfunktion `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Jetzt definieren wir die standardmäßige PyTorch-Trainingsschleife. Da unser Datensatz ziemlich groß ist, werden wir für unsere Lehrzwecke nur für eine Epoche trainieren und manchmal sogar weniger als eine Epoche (die Angabe des Parameters `epoch_size` ermöglicht es uns, das Training zu begrenzen). Wir würden auch die akkumulierte Trainingsgenauigkeit während des Trainings berichten; die Häufigkeit der Berichterstattung wird mit dem Parameter `report_freq` angegeben.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGrams, TriGrams und N-Grams

Eine Einschränkung des Bag-of-Words-Ansatzes ist, dass einige Wörter Teil von mehrwortigen Ausdrücken sind. Zum Beispiel hat das Wort 'Hot Dog' eine völlig andere Bedeutung als die Wörter 'hot' und 'dog' in anderen Kontexten. Wenn wir die Wörter 'hot' und 'dog' immer durch die gleichen Vektoren darstellen, kann das unser Modell verwirren.

Um dies zu lösen, werden **N-Gram-Darstellungen** häufig in Methoden der Dokumentklassifikation verwendet, bei denen die Häufigkeit jedes Wortes, Zwei-Wort- oder Drei-Wort-Ausdrucks ein nützliches Merkmal für das Training von Klassifikatoren ist. In der Bigram-Darstellung fügen wir beispielsweise alle Wortpaare zusätzlich zu den ursprünglichen Wörtern dem Vokabular hinzu.

Unten ist ein Beispiel, wie man eine Bigram-Bag-of-Words-Darstellung mit Scikit Learn generiert:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Der Hauptnachteil des N-Gramm-Ansatzes ist, dass die Größe des Vokabulars extrem schnell wächst. In der Praxis müssen wir die N-Gramm-Darstellung mit einigen Techniken zur Dimensionsreduktion kombinieren, wie zum Beispiel *Embeddings*, die wir in der nächsten Einheit besprechen werden.

Um die N-Gramm-Darstellung in unserem **AG News**-Datensatz zu verwenden, müssen wir ein spezielles N-Gramm-Vokabular erstellen:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Wir könnten denselben Code wie oben verwenden, um den Klassifikator zu trainieren, allerdings wäre das sehr speicherineffizient. In der nächsten Einheit werden wir einen Bigramm-Klassifikator mithilfe von Embeddings trainieren.

> **Hinweis:** Du kannst nur die N-Gramme beibehalten, die im Text häufiger als eine bestimmte Anzahl vorkommen. Das stellt sicher, dass seltene Bigramme ausgelassen werden und die Dimensionalität erheblich reduziert wird. Um dies zu erreichen, setze den Parameter `min_freq` auf einen höheren Wert und beobachte, wie sich die Länge des Vokabulars verändert.


## Termfrequenz-Inverse Dokumentfrequenz TF-IDF

In der BoW-Darstellung werden Wortvorkommen gleichmäßig gewichtet, unabhängig vom Wort selbst. Es ist jedoch offensichtlich, dass häufige Wörter wie *ein*, *in* usw. für die Klassifikation viel weniger wichtig sind als spezialisierte Begriffe. Tatsächlich sind bei den meisten NLP-Aufgaben einige Wörter relevanter als andere.

**TF-IDF** steht für **Termfrequenz–Inverse Dokumentfrequenz**. Es ist eine Variation des Bag-of-Words-Modells, bei der anstelle eines binären 0/1-Wertes, der das Auftreten eines Wortes in einem Dokument anzeigt, ein Gleitkommawert verwendet wird, der mit der Häufigkeit des Wortvorkommens im Korpus zusammenhängt.

Formal wird das Gewicht $w_{ij}$ eines Wortes $i$ im Dokument $j$ wie folgt definiert:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
wobei
* $tf_{ij}$ die Anzahl der Vorkommen von $i$ in $j$ ist, also der BoW-Wert, den wir zuvor gesehen haben
* $N$ die Anzahl der Dokumente in der Sammlung ist
* $df_i$ die Anzahl der Dokumente ist, die das Wort $i$ in der gesamten Sammlung enthalten

Der TF-IDF-Wert $w_{ij}$ steigt proportional zur Häufigkeit, mit der ein Wort in einem Dokument erscheint, und wird durch die Anzahl der Dokumente im Korpus, die das Wort enthalten, ausgeglichen. Dies hilft, den Umstand zu berücksichtigen, dass einige Wörter häufiger vorkommen als andere. Wenn beispielsweise ein Wort in *jedem* Dokument der Sammlung vorkommt, gilt $df_i=N$, und $w_{ij}=0$, und diese Begriffe würden vollständig ignoriert.

Mit Scikit Learn können Sie ganz einfach eine TF-IDF-Vektorisierung von Text erstellen:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Fazit

Auch wenn TF-IDF-Darstellungen Wörtern unterschiedliche Gewichtungen basierend auf ihrer Häufigkeit zuweisen, sind sie nicht in der Lage, Bedeutung oder Reihenfolge darzustellen. Wie der berühmte Linguist J. R. Firth 1935 sagte: „Die vollständige Bedeutung eines Wortes ist immer kontextabhängig, und keine Untersuchung der Bedeutung ohne Kontext kann ernst genommen werden.“ Später im Kurs werden wir lernen, wie man kontextuelle Informationen aus Texten mithilfe von Sprachmodellen erfasst.



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
